# Introduction to Web Scraping with Python

This notebook introduces the basic tools for web scraping with Python:

- What is web scraping?
- Accessing a webpage
- Extracting source code from a webpage (HTML)
- Parsing and navigating HTML with `BeautifulSoup`

## What is web scraping?

"Web scraping" is an umbrella term for (mostly) automated techniques for collecting information from the web (usually refers to collection *not* done manually in a browser).

Crawling, scraping and spiders are all various forms of scraping. Programs/scripts written for scraping can also be refered to as robots.

Working with web scraping involves the collection of *raw data* (source code) from the web as well as handling and converting these data to a manageable and analyzable format.

## The "legality" of web scraping

### Web scraping and copyright

The information and material on company and organisation websites are owned by those companies and organisations. A lot of websites have terms of use that prohibit the use of scraping or automated methods of collection on their websites. This is partly due to the fact that one can create a website with scrapers that duplicate the content of other websites. Even though this may not be our intention with the scraping that we do, it will often be considered a violation of the website's terms.

### Web scraping and personal data

Data on social media are a particular grey area when it comes to personal data. Is data that people voluntarily make available on public social media profiles still their data? A website like Twitter for example clearly states that all data made available on their platform is to be considered public information. Even so, a case could still be made that the data is still personal; just publicly available personal data.

Websites like Facebook, Twitter and Instagram also have terms of use for their data as they are responsible for it. Facebook and Instagram for exmaple do not allow any forms of automated collection of data.

### Web scraping og "hacking"

A website is located on a server. Each time a website is visited, a server is receiving a request to be processed. The more requests, the more busy the server is. Python allows us to easily write commands that send an incredible amount of requests within a very short time.

*This can easily be considered an attack and an attempt to congest a server which is illegal!*

## Accessing the internet with Python

The package `requests` can be used to send requests over the internet. 

When visiting a webpage, you are sending a "get" request to the server where the webpage is hosted. 

In Python, a get request can be send with `requests.get(url)`. This returns a request object (or a class) containing various attributes like the status code, headers and content.

In the code below, we send a request to the news overview for the EU's Climate Action section (https://ec.europa.eu/clima/news_en).

In [ ]:
import requests # Importing the package

response = requests.get("https://ec.europa.eu/clima/news_en")

`response` is now a request object containing various information of that request.

### Checking the request

To check if the request was successful, we can check the status code by inspecting the attribute `.status_code`:

In [ ]:
response.status_code

Status code 200 means "OK"; that our request was succesul. 

This can be verified by checking the attribute `.reason`:

In [ ]:
response.reason

In [ ]:
print(response.status_code, response.reason)

**Quick note on status codes**

- Status codes beginning with 2 or 3: The request is successful
- Status codes beginning with 4: The request has failed (client-side, fx 404 when specifying a URL that does not exist on a given domain).
- Status codes beginning with 5: The request has failed (server-side)

Status codes can be used in code to check whether or not a site is reached before scraping.

### Content of a webpage

The raw source code from a webpage can be extracted from the attribute `.content`.

In [ ]:
content = response.content
print(content[0:1000]) # Printing first 1000 characters

With this raw source code, one *could* process this as is using something like regular expression to find the relevant parts of the source code.

However, HTML has a certain structure. This can be utilized to extract specific information from a webpage.

## A quick introduction to HTML

Instead of processing the HTML as raw text, we can utilize the structure of HTML to extract specific parts of a webpage.

This requires some knowledge of what HTML is and how it is structured.

HTML is short for "Hyper-Text Markup Language". It is used on webpages to give the pages their structure.

HTML is structured in "tags" denoted by `<>` and `</>`. The tags denote what kind of content it is. `<p>` is for example a paragraph tag. A piece of HTML like: `<p> This is a paragraph </p>` will render the sentence "This is a paragraph" as a paragraph. Common tags include `h1` for headings (and `h2`, `h3` and so on), `a` for links and `div` for a "division" or "section".

HTML is structured in a tree-like structure. Tags are therefore usually located within other tags. Tags on the same level are refered to as "siblings", tags inside other tags are refered to as "children" and tags outside other tags are refered to as "parents".

HTML uses "attributes" to both differentiate between the same type of tags and to add other variables/information to the tag. The `id` attribute is fx used to give several tags a common id. `class` is used to differentiate between different tags and provide them with different stylings. A common and useful attribute is `href` which contain the link that a hyperlink is refering to.

```
    <html>
        <body>
            <div id="convo1">
                <p class="kenobi">Hello There!</p>
            </div>
            <div id="convo2">
                <p class="grievous">General Kenobi!</p>
            </div>
            <div id="convo3">
                <p class="kenobi">So Uncivilized!</p>
            </div>
        </body>
    </html>
```    


The code above is an example of HTML code. Rendered as a webpage it would only contain the text within the tags:

```
Hello There!

General Kenobi!

So Uncivilized!
```

The structure and the tags of the HTML allows us to extract only specific parts of the code. This is because the structure and the tags makes certain part of the code uniquely identifiable. For example:

- The text "Hello There!" is located within a p tag with the class "kenobi". 
- The p tag containing the text "Hello There!" is located within the div tag with id "convo1" (tags located inside other tags are refered to as "children")
- The div tag with id "convo1" is located next to another div tag with id "convo2" (tags located next to each other or on the same level are refered to as "siblings")

Combining the information, we can uniquely refer to the tag containing "Hello There!" by specifying that we want a p tag with class "kenobi" that is a child of a div tag with id "convo1".

## Parsing HTML with BeautifulSoup

The package "BeautifulSoup" (https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is developed specifically to navigate and parsing HTML (and XML) code. It works by converting HTML code to a "soup-object" wherein specific parts of the HTML can be extracted by refering to specific tags or paths.

The code below converts the HTML from before to a soup object using the function `bs`, which is a shorthand for the function `BeautifulSoup` imported from `bs4`:

In [ ]:
from bs4 import BeautifulSoup as bs

html = '<html><body><div id="convo1"><p class="kenobi">Hello There!</p></div><div id="convo2"><p class="grievous">General Kenobi!</p></div><div id="convo3"><p class="kenobi">So Uncivilized!</p></div></body></html>'
soup = bs(html, "html.parser") # The second arguement specifies the parser to use; how the code should be interpreted

In [ ]:
print(soup.prettify()) # Prints the HTML

When printed with `.prettify()` it looks like the same text but we are now able to navigate it using the tags.

### Finding tags

The methods `.find()` and `.find_all()` are used to find the first match and all matches respectively. The first argument of the method is the tag. Other arguments can then be added to make the search more specific.

Note that `.find()` and `.find_all()` are methods tied to a soup object, so they have to be used with some object returned from `bs` (in this case the object `soup` created earlier).

In [ ]:
soup.find("p") # Finds the first p tag

`.find()` returns a new soup object with the HTML in the first matched tag.

In [ ]:
soup.find_all("p") # Finds all p tags (returned as a list)

`.find_all()` returns a list of soup objects with the HTML in the matched tags.

The method `.get_text()` extracts the actual textual content within the tag (between `<p>` and `</p>` in this case):

In [ ]:
soup.find("p").get_text()

`.get_text()` works on a soup object and therefore not on returns from `find_all()`, as that returns a list. To extract the text from the contents of a list returned from `find_all()`, we have to iterate over the list elements (fx with a for loop):

In [ ]:
for tag in soup.find_all("p"):
    print(tag.get_text())

### Using attributes to find tags

In addition to searching for tags, we can also specify attributes. Some attributes have arguments specific for them like id and class.

Search for a div tag with a specific id attribute:

In [ ]:
soup.find("div", id = "convo1").get_text() # Search for a specific id attribute

Notice that `.get_text()` extracts *all* text within the tag including text within child tags.

Search for class attribute (notice the `_` added to `class_` as the `class` name is reserved somewhere else in Python):

In [ ]:
soup.find("p", class_ = "kenobi").get_text() # Search for a specific class attribute

Tags can also be found by searching for the attribute alone:

In [ ]:
soup.find(class_ = "kenobi").get_text()

BeautifulSoup supports a wide range of attributes (id, href, class). There are however no real rules as to what attributes can be called in HTML. BeautifulSoup therefore supports searching for any attribute with the following syntax:

`attrs = {"attribute": "value"}`

In [ ]:
soup.find(attrs = {"class": "kenobi"}).get_text()

#### Knowledge check:

What tags or attributes can be used to extract the text "General Kenobi"?

In [ ]:
print(soup.prettify())

## Finding the right tags

Let us try applying some of these skills on the European Union Climate Action news section.

We already know how to get the HTML, so this just has to be converted to a soup object, and we are ready to go. 

We first send a GET request and store the response as an object. As a lot of things can go wrong with a request, we check whether the request actually got through.

In [ ]:
response = requests.get("https://ec.europa.eu/clima/news_en")

print(response.status_code, response.reason)

If the output from the code above says "200 OK", we are ready to create the soup object from the HTML of the website:

In [ ]:
eu_html = response.content

eu_soup = bs(eu_html, "html.parser")

Finding the right tags by just browing through raw HTML is not ideal.

Instead we can use our browser to help us find the parts of the webpage to extract. Almost all browsers has an "inspector tool" of some kind that allows one to inspect the source code of a webpage (shortcut `F12` for a lot of browsers on Windows and `Command-Option-I` for Safari on Mac).

## Extracting news headlines from EU Climate Action News

Inspecting the HTML of https://ec.europa.eu/clima/news_en, we see that the headlines are part of an "a" tag within a div tag with the class "ecl-content-item__title". This class seems to be unique for the headlines.

We can extract the first headline as follows:

In [ ]:
news_title_soup = eu_soup.find("div", class_ = "ecl-content-item__title").find("a") # The find methods are chained; first span, then a

print(news_title_soup.get_text())

Note that the div tag in question actually contains two classes: "ecl-content-item__title" and "ecl-u-type-heading-5" (among others). HTML class names cannot contain spaces, so when an HTML tag contains a class attribute that contains spaces, it is actually two classes. When specifying the class with `.find()` or `.find_all()`, we only have to specify one of them.

### Retrieving links

The headline is also a link. Links are almost always created as an "a" tag.

`news_title_soup` is currently the soup object with the "a" tag containing the headline. Supposing we want to collect the links to the articles to scrape the articles themselves, we can extract that directly from this soup object.

The URL linked is almost always stored as an "href" attribute in an "a" tag.

Attributes can be extracted directly from soup objects using `[attribute]`:

In [ ]:
news_title_soup['href']

Links can be either "absolute" or "relative". Absolute links contain the entire URL to access the page. A relative URL contains the path on the specific domain. 

In order to convert the output a both to a working URL, we have to add the main domain, which can be done via pasting:

In [ ]:
print("https://ec.europa.eu" + news_title_soup['href'])

### Extracting all headlines

Extracting all the titles will have to be done step-wise, as `.find_all()` cannot be chained the same way because `.find_all()` always returns a list.

In [ ]:
span_soup = eu_soup.find_all("div", class_ = "ecl-content-item__title")
news_titles_soup = [title_soup.find("a") for title_soup in span_soup]
news_titles_soup = list(filter(None, news_titles_soup)) # Filtering empty

for title in news_titles_soup:
    print(title.get_text())

We can also store the titles as a list:

In [ ]:
title_list = [title_soup.get_text() for title_soup in news_titles_soup]

for title in title_list:
    print(title)

### Extracting article links

In [ ]:
for title in news_titles_soup:
    print(title['href'])

Stored as a list of links:

In [ ]:
link_list = [title_soup['href'] for title_soup in news_titles_soup]

for link in link_list:
    print(link)

### Extracting dates

The dates are inside a "time" tag within the "datetime" attribute:

In [ ]:
dates_soup = eu_soup.find_all("time")

dates = [date_soup["datetime"] for date_soup in dates_soup]

for date in dates:
    print(date)

### Extracting summaries

The summaries are inside a div tag with the class "ecl-content-item__description". The summaries can therefore be extracted as follows:

In [ ]:
summaries_soup = eu_soup.find_all("div", class_="ecl-content-item__description")

summaries_text = [summary_soup.get_text() for summary_soup in summaries_soup]

for summary in summaries_text:
    print(summary + "\n")

### Collecting the data in a structured format

To make it easier to work with the data later on, we can extract the information and structure it in some sensible format.

In the following, the title, link, date, summary and image URL of each news article is stored as a dictionary (`article_dict`). The articles are gathered in a list.

This format is essentially a list of JSONs.

In [ ]:
article_rows_soup = eu_soup.find_all("article", class_ = "ecl-content-item")

article_list = []

for row in article_rows_soup:
    article_dict = {}
    
    article_title_soup = row.find("div", class_ = "ecl-content-item__title").find("a")
    article_title = article_title_soup.get_text()
    article_link = article_title_soup['href']
    
    article_date = row.find("time")["datetime"]
    
    article_summary_soup = row.find("div", class_ = "ecl-content-item__description")
    try:
        article_summary = article_summary_soup.get_text(strip = True)
    except:
        article_summary = ""
    
    article_dict['title'] = article_title
    article_dict['link'] = article_link
    article_dict['date'] = article_date
    article_dict['summary'] = article_summary
    
    article_list.append(article_dict)

Data is now stored as a list of dictionaries: Each list element is a dictionary with the keys title, link, date, summary and imgurl.

In [ ]:
article_list[0]

This format can be converted to a pandas data frame with `pd.DataFrame.from_records()`

In [ ]:
import pandas as pd

eu_df = pd.DataFrame.from_records(article_list)

In [ ]:
eu_df.head()

## Supplementary material on web scraping

### Expanding search using regex

Attribute values can be long and sometimes adhere to a structure, where we want to find all attributes starting with some value. 

Instead of passing an exact string match as an arguement for `.find()`, one can instead parse a compiled regular expression pattern to search for.

We will not fully explain regular expression here but put shortly, regular expressions is a syntax for writing patterns that can match text strings. Instead of searching specifically for "kenobi", one could search for a pattern like starting with "ken" (`"^ken"`), ends with "obi" (`".*obi$"`) or contains six letters (`"\w{6}"`).

Regular expressions can be compiled using `re.compile(pattern)`. This pattern can the be used in `.find()` and `.find_all()`.

In [ ]:
from bs4 import BeautifulSoup as bs

html = '<html><body><div id="convo1"><p class="kenobi">Hello There!</p></div><div id="convo2"><p class="grievous">General Kenobi!</p></div><div id="convo3"><p class="kenobi">So Uncivilized!</p></div></body></html>'
soup = bs(html, "html.parser") # The second arguement specifies the parser to use; how the code should be interpreted
print(soup.prettify()) # Prints the HTML

In [ ]:
import re

soup.find(class_=re.compile("^gri")) # Search for tags with a class attribute starting with "gri"

### Search for specific text

The `.find()` and `.find_all()` methods have a `string = ` arguement to search for specific strings in the text of the HTML. Regular expressions can be used here as well.

In [ ]:
soup.find(string = re.compile("Hello"))

### Navigating the HTML structure

Using `.find()` returns a new soup object (`.find_all()` a list of soup objects). Because these methods search for tags *within* the soup object, it is always child tags of the original soup that is returned.

This allows one to parse further by first specifying one tag and then another:

In [ ]:
soup_child = soup.find("div")

soup_grandchild = soup_child.find("p")

print(soup_grandchild)

It also allows one to navigate the structure, as the extracted soup objects maintains references to the HTML structure that it was extracted from.

Using `.parent`, one can locate the tag in which a certain tag is located:

In [ ]:
soup_child = soup.find("p", class_ = "kenobi")

print(soup_child)

print(soup_child.parent) # Returns the parent of soup_child (a div tag in this case)

You can also iterate over all parents (and grand parents, so to speak) with `.parents`:

In [ ]:
for parent in soup_child.parents:
    print(parent.name)

Using `.next_sibling` and `.previous_sibling` you can navigate between tags on the same level:

In [ ]:
soup_child = soup.find("div")

print(soup_child)

print(soup_child.next_sibling) # Returns the next tag on the same level as soup_child